# Basic Tools for Alhazen

> Simple tools to demonstrate utility and 'agentic' functionality. 

## Available Tools:

### Building, Describing, and Deleting Collections:
1. **AddCollectionFromEPMCTool**: Execute a query against the European PMC, creating a collection, downloading available full-text papers and saving expressions, items, and fragments to the database to denote that publication.
2. **DescribeCollectionCompositionTool**: Describe the composition of a collection in the database.
3. **DeleteCollectionTool**: Delete a collection from the database.

### Searching for papers in collections:
3. **List__Paged_Expressions_CollectionCTool**: Delete a collection from the database.

## Tools under development:
1. Develop queries across external data sources
2. Extract information from a collection using an LLM-based analysis to create Notes 
3. Filter a collection by an LLM-based analysis by tagging fragments with Notes
5. Report on the state of the database in terms of numbers of collections, expressions, items, and fragments
6. Prepare a report over a core research question by collecting a number of notes and synthesizing them into a report

In [ ]:
#| default_exp tools.basic

In [ ]:
#| export

import os

from alhazen.core import PromptTemplateRegistry, load_alhazen_tool_environment
from alhazen.utils.searchEngineUtils import download_full_text_paper_for_doi
from alhazen.utils.web_robot import *
from alhazen.schema_sqla import *
from alhazen.utils.ceifns_db import Ceifns_LiteratureDb

import json

from langchain.tools import StructuredTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

from langchain.agents import AgentType, initialize_agent
from langchain.agents.agent import RunnableAgent
from langchain.chains import LLMMathChain
from langchain.chat_models.base import BaseChatModel
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.llms.base import BaseLanguageModel
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser

from operator import itemgetter
import pandas as pd
from pyalex import Works

from sqlalchemy import create_engine, exists, func, or_, desc
from sqlalchemy.orm import sessionmaker, aliased

from tqdm import tqdm

from typing import Optional, Type

import uuid

In [ ]:
#| export

skc = aliased(ScientificKnowledgeCollection)
skc_hm = aliased(ScientificKnowledgeCollectionHasMembers)
ske = aliased(ScientificKnowledgeExpression)
ske_ha = aliased(ScientificKnowledgeExpressionHasAuthors)
ske_hr = aliased(ScientificKnowledgeExpressionHasRepresentation)
ski = aliased(ScientificKnowledgeItem)
ski_hp = aliased(ScientificKnowledgeItemHasPart)
skf = aliased(ScientificKnowledgeFragment)
aut_iao = aliased(AuthorIsAuthorOf)
aut = aliased(Author)
aut_aff = aliased(AuthorAffiliations)
org = aliased(Organization)

In [ ]:
#| export
# NOTE - Use LangChain's SQL_DATABASE TOOL AS A MODEL 
# https://github.com/langchain-ai/langchain/blob/535db72607c4ae308566ede4af65295967bb33a8/libs/community/langchain_community/tools/sql_database/tool.py

class AlhazenToolMixin(BaseModel):
    '''Base tool for interacting with an Alhazen CEIFNS (pron. 'SAI-FiNS') database 
    (CEIFNS = Collection-Expression-Item-Fragment-Note-Summary).'''

    db: Ceifns_LiteratureDb = Field(exclude=True)
    llm: Optional[BaseChatModel] = Field(exclude=True)
    slm: Optional[BaseChatModel] = Field(exclude=True)

    class Config:
        """Configuration for this pydantic object."""
        arbitrary_types_allowed = True

class IntrospectionToolMixin(BaseModel):
    '''Base class for providing information about Alhazen's capabilities.'''

    agent: Optional[RunnableAgent] = Field(exclude=True)

    class Config:
        """Configuration for this pydantic object."""
        arbitrary_types_allowed = True

In [ ]:
#| export

class IntrospectionToolSchema(BaseModel):
    input: str = Field(description="A hypothetical input to an agent.")

class IntrospectionTool(IntrospectionToolMixin, BaseTool): 
    name = "introspect"
    description = "This tool permits Alhazen to answer questions how its agent would plan to answer specific questions."
    args_schema: Type[IntrospectionToolSchema] = IntrospectionToolSchema

    def _set_agent(self, agent):
        self.agent = agent

    def _run(
        self,
        input: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # 1. Check to see if the agent was instantiated
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        if self.agent is None:
            raise Exception('Introspection tool not instantiated with agent')
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # 2. Run agent's plan method 
        #~~~~~~~~~~~~~~~~~~~~~~~~~~
        payload = {'intermediate_steps': [], 'input': input}
        output = self.agent.invoke(payload)

        return output

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("add_collection_from_epmc_query does not support async")

In [ ]:
#| export

class AddCollectionFromEPMCToolSchema(BaseModel):
    id: str = Field(description="A code that serves as the id of the collection we will add papers to.")
    name: str = Field(description="A human-readable name of the collection we will add papers to.")
    query: str = Field(description="A search query (using AND/OR/NOT statements) to be executed on the European PMC online literature repository for scientific publications.")

class AddCollectionFromEPMCTool(AlhazenToolMixin, BaseTool): 
    name = "add_collection_from_epmc_query"
    description = "This tool constructs a collection in the database from an external source based on a search query ."
    args_schema: Type[AddCollectionFromEPMCToolSchema] = AddCollectionFromEPMCToolSchema
    return_direct:bool = True

    def _run(
        self,
        id: str, 
        name: str,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        n_papers_added = 0

        try: 
            
            c = self.db.add_corpus_from_epmc_query(id, name, query, commit_this=False)

            if self.db.embed_model is not None:
                skes = self.db.list_unindexed_skes(id)
                self.db.embed_expression_list(skes)
            
            # Need to add provenance for the data we have added to the model. 
            c.provenance = json.dumps([{'action_type': self.name, 'action': {'id': id, 'name': name, 'query': query} }])
            self.db.session.flush()

            #    if full_text:
            #        path = loc+db_name+'/ft/'
            #        ft_exist = download_full_text_paper_for_doi(doi, path)
            #        if ft_exist:
            #            self.db.add_full_text_for_expression(e)
            
            expression_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) 
            expression_count = expression_q.first()[0]

            jats_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'JATSFullText')
            jats_ft_count = jats_ft_q.first()[0]
            
            pdf_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'PDFFullText')
            pdf_ft_count = pdf_ft_q.first()[0]

            n = Note(id=uuid.uuid4().hex[0:10],
                    name='skc:%s.counts'%(id),
                    content='{"ske_count": %s, "jats_ft_count": %s, "pdf_ft_count": %s}' \
                        %(expression_count, jats_ft_count, pdf_ft_count),
                    format='json',
                    type='NoteAboutCollection')
            c.has_notes.append(n)        
            self.db.session.add(n)

            self.db.session.commit()
            
        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'response': 'We added a collection to the database called `{}` containing {} papers from this query: `{}`.'.format(name, n_papers_added, query)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("add_collection_from_epmc_query does not support async")

In [ ]:
#| export

class ListCollectionsToolSchema(BaseModel):
    name_filter: str = Field(description="A search string to filter the names of collections by.")

class ListCollectionsTool(AlhazenToolMixin, BaseTool): 
    name = "list_collections"
    description = "This tool lists available collections in the database."
    args_schema: Type[ListCollectionsToolSchema] = ListCollectionsToolSchema
    return_direct:bool = True

    def _run(
        self,
        search_string: str = '', 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        try: 
            if search_string != '':
                collections = self.db.session.query(skc) \
                    .filter(skc.name.like('%'+search_string+'%')).all()
            else:
                collections = self.db.session.query(skc).all()

        except Exception as e:
            return {'response': 'Error, cannot list available collections.', 'data': []}

        return {'response': 'Tool listed available {} collections.'.format(len(collections)),
                'data': collections}

    async def _arun(
        self,
        search_string: str = '', 
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("delete_collection does not support async")

In [ ]:
#| export

class AddCollectionFromOpenAlexToolSchema(BaseModel):
    id: str = Field(description="A code that serves as the id of the collection we will add papers to.")
    name: str = Field(description="A human-readable name of the collection we will add papers to.")
    dois: List[str] = Field(description="A list of dois to be added separately to the named collection.")

class AddCollectionFromOpenAlexTool(AlhazenToolMixin, BaseTool): 
    name = "add_collection_from_openalex_dois"
    description = "This tool constructs a collection in the database from an external source based on a list of dois."
    args_schema: Type[AddCollectionFromOpenAlexToolSchema] = AddCollectionFromOpenAlexToolSchema
    return_direct:bool = True

    def _run(
        self,
        id: str, 
        name: str,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        n_papers_added = 0

        try: 
            
            c = self.db.add_corpus_from_epmc_query(id, name, query, commit_this=False)
            
            # Need to add provenance for the data we have added to the model. 
            c.provenance = json.dumps([{'action_type': self.name, 'action': {'id': id, 'name': name, 'query': query} }])
            self.db.session.flush()

            #    if full_text:
            #        path = loc+db_name+'/ft/'
            #        ft_exist = download_full_text_paper_for_doi(doi, path)
            #        if ft_exist:
            #            self.db.add_full_text_for_expression(e)
            
            expression_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) 
            expression_count = expression_q.first()[0]

            jats_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'JATSFullText')
            jats_ft_count = jats_ft_q.first()[0]
            
            pdf_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'PDFFullText')
            pdf_ft_count = pdf_ft_q.first()[0]

            n = Note(id=uuid.uuid4().hex[0:10],
                    name='skc:%s.counts'%(id),
                    content='{"ske_count": %s, "jats_ft_count": %s, "pdf_ft_count": %s}' \
                        %(expression_count, jats_ft_count, pdf_ft_count),
                    format='json',
                    type='NoteAboutCollection')
            c.has_notes.append(n)        
            self.db.session.add(n)

            self.db.session.commit()
            
        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'response': 'We added a collection to the database called `{}` containing {} papers from this query: `{}`.'.format(name, n_papers_added, query)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("add_collection_from_epmc_query does not support async")

In [ ]:
#| export

class AddAuthorsToCollectionToolSchema(BaseModel):
    id: str = Field(description="A code that serves as the id of the collection we will add papers to.")

class AddAuthorsToCollectionTool(AlhazenToolMixin, BaseTool): 
    name = "add_authors_to_collection_query"
    description = "This tool adds authors to a collection."
    args_schema: Type[AddAuthorsToCollectionToolSchema] = AddAuthorsToCollectionToolSchema
    return_direct:bool = True

    def _run(
        self,
        id: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        n_papers_added = 0

        try: 
            
            print('Adding authors')
            for e in tqdm(self.db.list_expressions(id)):
        
                # Adding authors from open alex to build this. 
                # NOTE THAT THERE IS A 100,000 PER DAY TO LIMIT 
                # TO QUERY OPEN-ALEX WITHOUT A PAID LICENSE.
                # UGLY HACK TO ADD AUTHORS. 
                try:
                    doi = e.id.replace('doi:', 'https://doi.org/')
                    w = Works()[doi]
                    for aa in w['authorships']:
                        #~~~~~~~~~~~~~~~~~~~~~~~~~~~
                        # Add the author information 
                        #~~~~~~~~~~~~~~~~~~~~~~~~~~~
                        raw_a_name = aa.get('raw_author_name')
                        a = aa.get('author', {})
                        a_id = a.get('id', '')
                        a_name = a.get('display_name', '')
                        orcid = a.get('orcid')
                        q3 = self.db.session.query(aut).filter(aut.id == a_id)
                        author_ice = None
                        for temp_a in q3.all():
                            author_ice = temp_a
                            break
                        if author_ice is None:
                            author_ice = Author(id=a_id, name=a_name, type='Author')
                            self.db.session.add(author_ice)
                            if orcid:
                                author_ice.xref.append(orcid)
                        q4 = self.db.session.query(ske_ha) \
                            .filter(ske_ha.ScientificKnowledgeExpression_id == e.id) \
                            .filter(ske_ha.has_authors_id == a_id)
                        if len(q4.all()) == 0:
                            e.has_authors.append(author_ice)
                            author_ice.is_author_of.append(e)
                        
                        #~~~~~~~~~~~~~~~~~~~~~~~
                        # Add paper affiliations 
                        #~~~~~~~~~~~~~~~~~~~~~~~
                        raw_inst_names =  aa.get('raw_affiliation_names')
                        for inst in aa.get('institutions', []):
                            inst_id = inst.get('id', '')
                            inst_country = inst.get('country_code', '')
                            inst_name = inst.get('display_name', '')
                            ror = inst.get('ror')
                        
                            q5 = self.db.session.query(org).filter(org.id == inst_id)
                            org_ice = None
                            for oo in q5.all():
                                org_ice = oo
                                break
                            if org_ice is None:
                                q5a = self.db.session.query(Country) \
                                    .filter(Country.code2 == inst_country)
                                org_ice = Organization(id=inst_id, 
                                                       name=inst_name,
                                                       type='Organization',
                                                       country=[c for c in q5a.all()])
                                self.db.session.add(org_ice)
                                if ror:
                                    org_ice.xref.append(ror)
                            q6 = self.db.session.query(aut_aff) \
                                .filter(aut_aff.Author_id == a_id) \
                                .filter(aut_aff.affiliations_id == inst_id)
                            if len(q6.all()) == 0:
                                author_ice.affiliations.append(org_ice)
                    self.db.session.flush()
                
                except Exception as exc: 
                    print(exc)
                    print('errors with adding author information via OpenAlex')
            
        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'response': 'We added authors and institutions to the collection with the id=`{}`.'.format(id)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("add_collection_from_epmc_query does not support async")

In [ ]:
#| export

class CheckExpressionToolSchema(BaseModel):
    query: str = Field(description="a search query for the paper in the database")
    
class CheckExpressionTool(AlhazenToolMixin, BaseTool): 
    name = "describe_expression_in_local_database"
    description = "This tool searches for a paper in the database and reports if we have full text version of it."
    args_schema: Type[CheckExpressionToolSchema] = CheckExpressionToolSchema
    return_direct:bool = True

    def _run(
        self,
        query: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        try:      
            ske = aliased(ScientificKnowledgeExpression)
            ske_hr = aliased(ScientificKnowledgeExpressionHasRepresentation)
            ski = aliased(ScientificKnowledgeItem)

            expression_q = self.db.session.query(ske.id, ske.content, ske.type, func.aggregate_strings(ski.type, ',')) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(or_(ske.id.like('%'+query+'%'), ske.content.like('%'+query+'%'))) \
                    .group_by(ske.id, ske.content, ske.type)
            df = pd.DataFrame(expression_q.all(), columns=['e_id', 'citation', 'e_type', 'i_type'])
            
        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        if len(df) == 0:
            return {'response': 'Could not find a match to query `{}`'.format(query)}
        elif len(df) > 1:
            return {'response': 'Found more than one match to query `{}`'.format(query)}
            
        p_id = None
        ft_available = False
        for i, row in df.iterrows():
            if 'FullText' in row.i_type:
                p_id = row.e_id
                ft_available = True

        return {'response': 'Found one match to query `{}` with paper_id={}, full_text_available={}'.format(query, p_id, ft_available)}
        
    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("describe_expression_in_local_database does not support async")

In [ ]:
#| export

class DescribeCollectionCompositionToolSchema(BaseModel):
    id: str = Field(description="should be the id of the collection we wish to describe")
    
class DescribeCollectionCompositionTool(AlhazenToolMixin, BaseTool): 
    name = "describe_collection_in_local_database"
    description = "This tool describes the contents of a Collection, in terms of counts of papers"
    args_schema: Type[DescribeCollectionCompositionToolSchema] = DescribeCollectionCompositionToolSchema
    return_direct:bool = True

    def _run(
        self,
        id: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        try:      
            collection_q = self.db.session.query(skc.name) \
                    .filter(skc.id == str(id)) 
            df = pd.DataFrame(collection_q.all(), columns=['collection_name'])
            if len(df) == 0:
                return {'response': 'The collection with the id `{}` does not exist.'.format(id)}

            name = collection_q.first()[0]

            expression_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) 
            expression_count = expression_q.first()[0]

            jats_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'JATSFullText')
            jats_ft_count = jats_ft_q.first()[0]

            pdf_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'PDFFullText')
            pdf_ft_count = pdf_ft_q.first()[0]

        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'response': 'The collection with the id `{}` is called `{}`. It contains {} papers; including {} PDF and {} JATS full text papers.'.format(id, name, expression_count, pdf_ft_count, jats_ft_count)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("add_collection_from_epmc_query does not support async")

In [ ]:
#| export

class DeleteCollectionToolSchema(BaseModel):
    collection_id: str = Field(description="should be the collection_id of the collection we will add papers to")

class DeleteCollectionTool(AlhazenToolMixin, BaseTool): 
    name = "delete_collection"
    description = "This deletes a collection from the database based on an collection_id value."
    args_schema: Type[DeleteCollectionToolSchema] = DeleteCollectionToolSchema
    return_direct:bool = True

    def _run(
        self,
        collection_id: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        try:             
            self.db.delete_collection(collection_id, commit_this=True)
        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'response': 'Successfully deleted a collection with collection_id:`{}`.'.format(collection_id)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("delete_collection does not support async")

In [ ]:
#| export

class RetrieveFullTextToolForACollectionSchema(BaseModel):
    collection_id: str = Field(description="the identifier of the collection containing papers to be retrieved")

class RetrieveFullTextToolForACollection(AlhazenToolMixin, BaseTool): 
    name = "retrieve_full_text_for_papers_in_collection"
    description = "This retrieves all full text papers in a given collection."
    args_schema: Type[RetrieveFullTextToolForACollectionSchema] = RetrieveFullTextToolForACollectionSchema
    return_direct:bool = True

    def _run(
        self,
        collection_id: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        loc = load_alhazen_tool_environment()
        db_name = self.db.name

        skc = aliased(ScientificKnowledgeCollection)
        for c in self.db.session.query(skc) \
                .filter(skc.id == str(collection_id)).all():
            break 
        
        ske = aliased(ScientificKnowledgeExpression)
        exp_q = self.db.session.query(ske) \
                .filter(skc_hm.has_members_id == ske.id) \
                .filter(skc_hm.ScientificKnowledgeCollection_id == str(collection_id)) \
                .order_by(desc(ske.publication_date))
        for e in tqdm(exp_q.all()):
            doi = e.id.replace('doi:', '')
            path = loc+db_name+'/ft/'

            # Check if we have already downloaded the full text for the paper
            check = self.db.check_full_text(e)
            if check.get('nxml_present') is False and \
                    check.get('pdf_present') is False and \
                    check.get('html_present') is False and \
                    check.get('prior_attempts') is False:
                try:
                    ft_exist = download_full_text_paper_for_doi(doi, path, headless=True)  
                    if ft_exist: 
                        self.db.write_note_about_x(e, 'successfully downloaded full text for %s'%(e.id), '', 'log', 'NoteAboutExpression', commit_this=True)
                    else:
                        self.db.write_note_about_x(e, 'failed to download full text for %s'%(e.id), '', 'log', 'NoteAboutExpression', commit_this=True)                    
                except Exception as ex:
                    print(ex)
                    content = repr(ex)
                    self.db.write_note_about_x(e, 'errored whilst downloading full text for %s'%(e.id), content, 'log', 'NoteAboutExpression', commit_this=True)                    
     
            check2 = self.db.check_full_text(e)
            if check2.get('nxml_present') or check2.get('pdf_present') or check2.get('html_present') and \
                (check2.get('pdf_item_present') is False and check2.get('nxml_item_present') is False):
                try:
                    self.db.add_full_text_for_expression(e)
                    self.db.session.commit()

                    check3 = self.db.check_full_text(e)
                    if check3.get('pdf_item_present') or check3.get('nxml_item_present'):
                        self.db.write_note_about_x(e, 'successfully added full text for %s'%(e.id), '', 'log', 'NoteAboutExpression', commit_this=True)
                    else:
                        self.db.write_note_about_x(e, 'failed to add full text for %s'%(e.id), '', 'log', 'NoteAboutExpression', commit_this=True)
                except Exception as ex:
                    print(ex)
                    self.db.session.rollback()
                    content = repr(ex)
                    self.db.write_note_about_x(e, 'errored whilst adding full text for %s'%(e.id), content, 'log', 'NoteAboutExpression', commit_this=True)
            
        return {'response': 'I retrieved full text papers for the collection named `{}` (id=`{}).'.format(c.name, c.id)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("retrieve_full_text_for_papers_in_collection does not support async")

In [ ]:
#| export

class RetrieveFullTextToolSchema(BaseModel):
    paper_id: str = Field(description="the digitial objecty identifier (doi) of the paper being retrieved from external sources")

class RetrieveFullTextTool(AlhazenToolMixin, BaseTool): 
    name = "retrieve_full_text_for_paper_id"
    description = "This retrieves a full text paper based on its doi, copies it to local disk, and adds it the database."
    args_schema: Type[RetrieveFullTextToolSchema] = RetrieveFullTextToolSchema
    return_direct:bool = True

    def _run(
        self,
        paper_id: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        loc = load_alhazen_tool_environment()
        db_name = self.db.name

        try: 
            ske = aliased(ScientificKnowledgeExpression)
            e = self.db.session.query(ske) \
                    .filter(ske.id == str(paper_id)).first()
            doi = e.id.replace('doi:', '')
            path = loc+db_name+'/ft/'

                        # Check if we have already downloaded the full text for the paper
            check = self.db.check_full_text(e)
            if check.get('nxml_present') is False and \
                    check.get('pdf_present') is False and \
                    check.get('html_present') is False and \
                    check.get('prior_attempts') is False:
                try:
                    ft_exist = download_full_text_paper_for_doi(doi, path, headless=True)  
                    if ft_exist: 
                        self.db.write_note_about_x(e, 'successfully downloaded full text for %s'%(e.id), '', 'log', 'NoteAboutExpression', commit_this=True)
                    else:
                        self.db.write_note_about_x(e, 'failed to download full text for %s'%(e.id), '', 'log', 'NoteAboutExpression', commit_this=True)                    
                except Exception as ex:
                    print(ex)
                    self.db.session.rollback()
                    content = repr(ex)
                    self.db.write_note_about_x(e, 'errored whilst downloading full text for %s'%(e.id), content, 'log', 'NoteAboutExpression', commit_this=True)                    
     
            check2 = self.db.check_full_text(e)
            if check2.get('nxml_present') or check2.get('pdf_present') or check2.get('html_present') and \
                (check2.get('pdf_item_present') is False and check2.get('nxml_item_present') is False):
                try:
                    self.db.add_full_text_for_expression(e)
                    self.db.session.commit()

                    check3 = self.db.check_full_text(e)
                    if check3.get('pdf_item_present') or check3.get('nxml_item_present'):
                        self.db.write_note_about_x(e, 'successfully added full text for %s'%(e.id), '', 'log', 'NoteAboutExpression', commit_this=True)
                    else:
                        self.db.write_note_about_x(e, 'failed to add full text for %s'%(e.id), '', 'log', 'NoteAboutExpression', commit_this=True)
                except Exception as ex:
                    print(ex)
                    self.db.session.rollback()
                    content = repr(ex)
                    self.db.write_note_about_x(e, 'errored whilst adding full text for %s'%(e.id), content, 'log', 'NoteAboutExpression', commit_this=True)

        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'response': 'The full text paper with doi:`{}` is available in the database.'.format(paper_id)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("delete_collection does not support async")